In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Install Dependencies
!pip install ultralytics opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [4]:
# Imports
import cv2
import torch
from ultralytics import YOLO
import numpy as np
import os

# 🔍 Load YOLOv5 model
model = YOLO('yolov5s.pt')
model.conf = 0.5

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 17.7M/17.7M [00:00<00:00, 179MB/s]


In [7]:
# Set up paths
input_video_path = '/content/drive/MyDrive/dataset/coffee_mugs.mp4'
output_video_path = '/content/drive/MyDrive/dataset/mug_tracking_output.mp4'

# Load YOLOv5 model from Ultralytics
model = YOLO('yolov5s.pt')
model.conf = 0.5

# Open video file
cap = cv2.VideoCapture(input_video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Video Writer for output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Tracker setup
multi_tracker = cv2.legacy.MultiTracker_create()
tracker_type = 'csrt'
mug_ids = {}
initialized = False

# Create OpenCV tracker
def create_tracker():
    return cv2.legacy.TrackerCSRT_create()

PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [8]:
# Run detection + tracking
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if not initialized:
        results = model(frame)[0]
        count = 0
        for det in results.boxes:
            cls_id = int(det.cls[0].item())
            conf = det.conf[0].item()
            label = model.names[cls_id]
            if label == 'cup':
                x1, y1, x2, y2 = map(int, det.xyxy[0].tolist())
                bbox = (x1, y1, x2 - x1, y2 - y1)
                tracker = create_tracker()
                multi_tracker.add(tracker, frame, bbox)
                mug_ids[len(mug_ids)] = bbox
                count += 1
                if count == 2:
                    break
        initialized = True
    else:
        success, boxes = multi_tracker.update(frame)
        for i, newbox in enumerate(boxes):
            (x, y, w, h) = [int(v) for v in newbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"Mug ID: {i}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    out.write(frame)
    frame_count += 1

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Output video saved to: {output_video_path}")


0: 384x640 2 persons, 2 cups, 1 chair, 1 dining table, 44.3ms
Speed: 22.4ms preprocess, 44.3ms inference, 347.6ms postprocess per image at shape (1, 3, 384, 640)
Output video saved to: /content/drive/MyDrive/dataset/mug_tracking_output.mp4
